This notebook computes the learning outcomes from the pre-test and post-test responses.

We consider two learning goals:
* LG1: identify a (feasible) solution to the problem
* LG2: construct a correct (i.e. optimal) solution.
    
For LG1, we check connectedness, i.e. if solutions are spanning or not
out.

For LG2, we check optimality, i.e. how close was the solution to the correct solution, quantified by the error.

In [1]:
import math
import copy
import pickle

import pathlib as pl
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from scipy.stats import wilcoxon, shapiro, ttest_rel
from effsize.effsize import two_group_difference

### Define paths.

In [2]:
transition_tables_pickle_file = pl.Path(
    '../processed_data/justhink_spring21_transition_tables.pickle')

learning_pickle_file = pl.Path(
    '../processed_data/learning_table.pickle')

### Load transition tables.

In [3]:
with transition_tables_pickle_file.open('rb') as handle:
    transition_tables = pickle.load(handle)

In [9]:
# transition_tables.keys()
# pd.options.display.max_rows = None
# transition_tables[10]

<!-- ### Cleaning tables. -->

### Fiter for the submissions.

In [8]:
def filter_submissions(transition_tables, verbose=False):
    """Get the submission states only, and drop the duplicates, 
    unused columns, keep state, activity, imported info."""
    tables = {}
    
    for participant in sorted(transition_tables):
        print('Processing for participant {}...'.format(participant))
        
        df = transition_tables[participant].copy()
        
        # Filter for the submission rows.
        df = df[df['is_submission']]

        # Remove. collaborative activity rows
        # df = df[df['header.frame_id'] != "collab-activity"]
        # df = df[df['header.frame_id'] != "collab-activity-2"]

        # Remove duplicate rows from submission log and keeping the last submission
        df.drop_duplicates(subset='activity', keep='last', inplace=True)

        mst_costs = []
        spanning = []
        norm_error = []

        for i, row in df.iterrows():
            cost = row['state'].network.get_mst_cost()
            span = row['state'].network.is_spanning()
            mst_costs.append(cost)
            spanning.append(span)
            # compute normalized error
            if span:
                opt_cost = row['state'].network.get_mst_cost()
                cost = row['cost']
                error = (cost - opt_cost) / opt_cost
                norm_error.append(error)
            else:
                norm_error.append(None)

        # adding mst_cost, spanning, and normalized_error columns
        df['mst_cost'] = mst_costs
        df['spanning'] = spanning
        df['error'] = norm_error

        tables[participant] = df

        if verbose:
            display(df)

    return tables


submission_tables = filter_submissions(transition_tables)

Processing for participant 1...


KeyError: Index(['header.frame_id'], dtype='object')

In [ ]:
all_tables[1]

### Check connectivity, optimality, and compute average error.

In [ ]:
def compute_spanning_score(df):
    """Calculate spanning score for pre-test and post-test."""
    pre_spanning_score = 0
    post_spanning_score = 0
    for index, row in df.iterrows():
        if 'pre' in row['header.frame_id']:
            if row['spanning']:
                pre_spanning_score += 1
        elif 'post' in row['header.frame_id']:
            if row['spanning']:
                post_spanning_score += 1
    return pre_spanning_score, post_spanning_score


for key, table in all_tables.items():
    print('participant', key)
    print('spanning scores: ', compute_spanning_score(table))

In [ ]:
def compute_mst_score(df):
    """Compute is_mst score for pre-test and post-test."""
    pre_mst_score = 0
    post_mst_score = 0
    for index, row in df.iterrows():
        if 'pre' in row['header.frame_id']:
            if row['is_mst']:
                pre_mst_score += 1
        elif 'post' in row['header.frame_id']:
            if row['is_mst']:
                post_mst_score += 1
    return pre_mst_score, post_mst_score

for key, table in all_tables.items():
    print('student', key)
    print('mst scores: ',compute_mst_score(table))

In [ ]:
def compute_average_error(df):
    """Compute average error for pre-test and post-test."""
    pre_total = 0
    post_total = 0
    pre_count = 0
    post_count = 0
    for index, row in df.iterrows():
        value = row['error']
        if 'pre' in row['header.frame_id']:
            # not computing the error for submissions that are not feasbile/spanning
            if not math.isnan(value):
                pre_total += value
                pre_count += 1
        elif 'post' in row['header.frame_id']:
            # not computing the error for submissions that are not feasbile/spanning
            if not math.isnan(value):
                post_total += value
                post_count += 1

    # compute averages while handling zero division
    if pre_count != 0:
        pre_avg = pre_total/pre_count
    else:
        pre_avg = None
    if post_count != 0:
        post_avg = post_total/post_count
    else:
        post_avg = None
    return pre_avg, post_avg


for participant, table in all_tables.items():
    print('participant {} pre- post- average errors = {}'.format(
        participant, compute_average_error(table)))

### Combine all outcomes into a summary table.

In [ ]:
learn_df = pd.DataFrame()
learn_df['participant'] = all_tables.keys()

pre_span = list()
post_span = list()
pre_mst = list()
post_mst = list()
pre_error = list()
post_error = list()

# computing scores and appending them to the master table
for participant, table in all_tables.items():
    pre_span_val, post_span_val = compute_spanning_score(table)
    pre_mst_val, post_mst_val = compute_mst_score(table)
    pre_error_avg, post_error_avg = compute_average_error(table)
    pre_span.append(pre_span_val)
    post_span.append(post_span_val)
    pre_mst.append(pre_mst_val)
    post_mst.append(post_mst_val)
    pre_error.append(pre_error_avg)
    post_error.append(post_error_avg)

learn_df['pretest_span'] = pre_span
learn_df['posttest_span'] = post_span
learn_df['pretest_mst'] = pre_mst
learn_df['posttest_mst'] = post_mst
learn_df['pre_error'] = pre_error
learn_df['post_error'] = post_error

# sorting table by participant.
learn_df.sort_values(by=['participant'], inplace=True)

display(learn_df)

#premin error
#pre max error

### Reshape tables for visualization.

In [ ]:
# reformatting is_spanning and is_mst dataframes for visualization
span_learning_df = learn_df[['participant', 'pretest_span', 'posttest_span']]
mst_learning_df = learn_df[['participant', 'pretest_mst', 'posttest_mst']]
error_learning_df = learn_df[['participant', 'pre_error', 'post_error']]

spandf = pd.DataFrame(np.repeat(span_learning_df.values, 2, axis=0))
spandf.columns = span_learning_df.columns

mstdf = pd.DataFrame(np.repeat(mst_learning_df.values, 2, axis=0))
mstdf.columns = mst_learning_df.columns

errordf = pd.DataFrame(np.repeat(error_learning_df.values, 2, axis=0))
errordf.columns = error_learning_df.columns

# reformatting spanning dataframe
p_type = list()
value = list()
for index, row in spandf.iterrows():
    if index % 2 == 0:
        p_type.append('pretest')
        value.append(row['pretest_span'])
    else:
        p_type.append('posttest')
        value.append(row['posttest_span'])
spandf['p_type'] = p_type
spandf['value'] = value

# reformatting mst dataframe
mst_value = list()
for index, row in mstdf.iterrows():
    if index % 2 == 0:
        mst_value.append(row['pretest_mst'])
    else:
        mst_value.append(row['posttest_mst'])
mstdf['p_type'] = p_type
mstdf['value'] = mst_value

# reformatting error dataframe
error_value = list()
for index, row in errordf.iterrows():
    if index % 2 == 0:
        error_value.append(row['pre_error'])
    else:
        error_value.append(row['post_error'])
errordf['p_type'] = p_type
errordf['value'] = error_value


spandf.drop(columns=['pretest_span', 'posttest_span'], inplace=True)
mstdf.drop(columns=['pretest_mst', 'posttest_mst'], inplace=True)
errordf.drop(columns=['pre_error', 'post_error'], inplace=True)

display(spandf)
display(mstdf)
display(errordf)

### Reshape collaborative activity table.

In [ ]:
# formatting dataframe for collab activities
cdf = pd.DataFrame()

for key, mdf in all_tables.items():
    collab_df = mdf.loc[mdf['header.frame_id'].isin(
        ['collab-activity', 'collab-activity-2'])].copy()
    collab_df['participant'] = key
    collab_df.drop(columns=['action.agent_name', 'state', 'is_submission',
                   'cost', 'mst_cost', 'spanning', 'error'], inplace=True)
    collab_df.rename(columns={"header.frame_id": "activity"}, inplace=True)
    cdf = cdf.append(collab_df)

cdf.sort_values(by=['participant'], inplace=True)
values = list()
for index, row in cdf.iterrows():
    if row['is_mst'] == True:
        values.append(1)
    else:
        values.append(0)
cdf['is_mst'] = values
cdf

### Visualize collaborative activity performance.

In [ ]:
participants = [1, 2, 3, 4, 5, 6, 7, 9, 10]
separation = 0.01

# graphing the pretest versus posttest is_spanning scores
fig, ax = plt.subplots(1, figsize=(5, 5))

prelist = list()
postlist = list()
for i in participants:
    temp = cdf[cdf['participant'] == i]
    values = list(temp.is_mst)
    #computing offset between overlapping lines
    p0 = values[0] + prelist.count(values[0])*separation
    p1 = values[1] + prelist.count(values[1])*separation
    prelist.append(values[0])
    postlist.append(values[1])
    #plotting
    plt.plot(temp.activity, [p0,p1], marker='o', markersize=5)
plt.ylabel('Score')
plt.title('Collab Activity Success\n', loc='center', fontsize=20)

leg = plt.legend(participants, loc='upper left', frameon=True)
# get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)
# change to location of the legend
xOffset = 1.1
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform=ax.transAxes)

# set y axis ticks to percentages
yticks = plt.yticks()[0]
plt.yticks(np.arange(0, 2, step=1), ['Fail','Success'])
plt.show()

### Visualize pretest and postest learning scores.

In [ ]:
participants = [1, 2, 3, 4, 5, 6, 7, 9, 10]
separation = 0.06

# graphing the pretest versus posttest is_spanning scores
fig, ax = plt.subplots(1, figsize=(5, 7))

prelist = list()
postlist = list()
for i in participants:
    temp = spandf[spandf['participant'] == i]
    values = list(temp.value)
    #computing offset between overlapping lines
    p0 = values[0] + prelist.count(values[0])*separation
    p1 = values[1] + prelist.count(values[1])*separation
    prelist.append(values[0])
    postlist.append(values[1])
    #plotting
    plt.plot(temp.p_type, [p0,p1], marker='o', markersize=5)
plt.ylabel('Score')
plt.title('Feasible Solution\n', loc='center', fontsize=20)


leg = plt.legend(participants, loc='upper left', frameon=True)
# get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)
# change to location of the legend
xOffset = 1.1
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform=ax.transAxes)

# set y axis ticks to percentages
yticks = plt.yticks()[0]
plt.yticks(yticks[1:-1], [str(round((i/5)*100)) + '%' for i in yticks[1:-1]])
plt.show()

# graphing the pretest versus posttest is_mst scores
fig, ax = plt.subplots(1, figsize=(5, 7))

prelist = list()
postlist = list()
for i in participants:
    temp = mstdf[mstdf['participant'] == i]
    values = list(temp.value)
    #computing offset between overlapping lines
    p0 = values[0] + prelist.count(values[0])*separation
    p1 = values[1] + prelist.count(values[1])*separation
    prelist.append(values[0])
    postlist.append(values[1])
    #plotting
    plt.plot(temp.p_type, [p0,p1], marker='o', markersize=5)
plt.ylabel('Score')
plt.title('Optimal Solution\n', loc='center', fontsize=20)

leg = plt.legend(participants, loc='upper left', frameon=True)
# get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)
# change to location of the legend
xOffset = 1.1
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform=ax.transAxes)

# set y axis ticks to percentages
yticks = plt.yticks()[0]
plt.yticks(yticks[1:-1], [str(round((i/5)*100)) + '%' for i in yticks[1:-1]])
plt.show()

# graphing the pretest versus posttest spanning scores
fig, ax = plt.subplots(1, figsize=(5, 7))
for i in participants:
    temp = errordf[errordf['participant'] == i]
    plt.plot(temp.p_type, temp.value, marker='o', markersize=5)
plt.ylabel('Score')
plt.title('Error Average\n', loc='center', fontsize=20)

leg = plt.legend(participants, loc='upper left', frameon=True)
# get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)
# change to location of the legend
xOffset = 1.1
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform=ax.transAxes)

# set y axis ticks to percentages
yticks = plt.yticks()[0]
plt.yticks(yticks[1:-1], [str(round(i*100)) + '%' for i in yticks[1:-1]])
plt.show()

### Perform Wilcoxon signed-rank test.

In [ ]:
df = learning_df.copy()
df.rename(columns={"pre_error": "pretest_error", "post_error":"posttest_error"}, inplace=True)
display(df)
df.dropna(inplace=True)

categories = ['span','mst', 'error']

for name in categories:
    r = list(df['pretest_' + name])
    o = list(df['posttest_' + name])
    print(name)
    w, p = wilcoxon(r,o,mode="exact") 
    #null hypotehsis says they are the same, p val less than threshold, reject hyp, conclude that post is larger than pretest
    print('Exact:','W=',w,'pvalue=',p)
    a,b = wilcoxon(r,o,mode="exact", alternative="greater")
    print('Greater:','W=',a,'pvalue=',b)

# display(learning_df)

In [ ]:
with learning_pickle_file.open('wb') as handle:
    pickle.dump(learn_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Saved learning table to {}'.format(learning_pickle_file))

### Compute the effect size (estimated by Cliff's delta).

In [ ]:
categories = ['span', 'mst', 'error']

for name in categories:
    r = list(df['pretest_{}'.format(name)])
    o = list(df['posttest_{}'.format(name)])
    print(name)
    
    # Estimate effect size by Cliff's Delta.
    d = two_group_difference(control=r, test=o, effect_size='cliffs_delta')
    print("For categury {} Cliff's delta = {}".format(name, d)